In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [35]:
url_inicial = 'https://www.revistabiomedica.mx/index.php/revbiomed/issue/archive?issuesPage=4#issues'
url_root = 'https://www.revistabiomedica.mx/index.php/revbiomed/issue/archive?issuesPage=4#issues'
r=requests.get(url_inicial)

In [36]:
soup = BeautifulSoup(r.text, 'html.parser')

In [37]:
box = soup.find('div', id='issues')
volumen=box.findAll('div', style='float: left; width: 100%;')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.revistabiomedica.mx/index.php/revbiomed/issue/view/32']

In [38]:
def get_url_items(sopa, url): 
    box = soup.find('div', id='issues')
    volumen=box.findAll('div', style='float: left; width: 100%;')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [39]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://www.revistabiomedica.mx/index.php/revbiomed/issue/archive?issuesPage=4#issues


In [40]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

1

In [41]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [42]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Biomédica'
    a='III. Medicina y Ciencias de la Salud'
    tem='Biomedica en General'
    url='http://revistabiomedica.mx/index.php/revbiomed'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('div', class_='tocTitle').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('a', class_='cover').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='file').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [43]:
list_scraper=list_scraper[0:9]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0


In [44]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Revista Biomédica,III. Medicina y Ciencias de la Salud,Biomedica en General,La Revista Biomédica: una década de publicación,None,http://revistabiomedica.mx/index.php/revbiomed,https://www.revistabiomedica.mx/index.php/revb...


In [45]:
df_catalogo.to_csv('Revista181.04.csv', index=False)